In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from rf import *

In [ ]:
'''
PARAMETERS: modify in notebook_params notebook, then run that notebook and this cell to update here
DO NOT modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n brdf_dir = {} \n gridCell = {} \n index_dir = {} \n home_dir = {}".format(basic_config['brdf_dir'],basic_config['grid_cell'],basic_config['index_dir'],basic_config['home_dir']))
%store -r classification_params
print("Classification_Params: \n model_type = {} \n samp_pt_file = {} \n pix_vars = {} \n pixdf = {} \n model_name = {} \n lc_class = {} \n ranhold = {} \n impmeth = {}".format(classification_params['model_type'],classification_params['samp_pts'],classification_params['pix_vars'],classification_params['pixdf'],classification_params['model_name'],
classification_params['lc_mod'],classification_params['ranhold'],classification_params['impmeth']))
      

## View the look up table
These are the different LC_models to group things in classification and to translate between numerical map categories and text labels

In [ ]:
lut=pd.read_csv('../Class_LUT.csv')
lut.drop(['Description'], axis=1, inplace=True)
print(lut.sort_values('LC_UNQ'))

## get sample dataframe:
 pixdf is the combination of the sample point file and variable stack for those points (pix_vars).
 This is created in notebook 6a_RandomForest_VariableDataframe

In [ ]:
pixdf = pd.read_csv(classification_params['pixdf'])
print('sample breakdown by {}:'.format(classification_params['lc_mod']))
label_col, new_lut = get_class_col(classification_params['lc_mod'], lut)
if '{}_name'.format(label_col) in pixdf.columns:
    print(pixdf['{}_name'.format(label_col)].value_counts())
else:
    pixdf2 = pixdf.merge(new_lut[['USE_NAME','{}'.format(label_col),'{}_name'.format(label_col)]], left_on='Class', right_on='USE_NAME', how='left')
    print(pixdf2['{}_name'.format(label_col)].value_counts())

## (optional): filter dataframe
(if using the same dataset repeatedly, use notebook 6a to filter and print the dataframe as a new pixdf)

by rejoining the pixel dataframe to ptdf, we can get attributes that can be used to filter cases.
e.g. "PurePixel"==X indicates fields that are on the edge and likely to be mixed with another class
"FieldWidth" gives an approximate field size (usally provided if fields are very small so to enable removal of small fields). "rand" is a random number from .00-.99 that can be used to reduce the dataset/stratify categories
"TESTSET10" and "TESTSET20" are flagged if a point is in a 10% or 20% (stratified) holdout group.
"SampMethod" has the sampling method (e.g. ground vs. Google Earth) 

In [ ]:
### example ###########################
#pixdf = pixdf[(pixdf['rand']>.5)]
#print(pixdf['LC17_name'].value_counts())

## create rf model
this uses the multiclass RandomForestClassifier method from sklearn.ensemble (code is in ../LUCinSA_helpers/rf.py)

To use a different classification model, change 'lc_mod' in the parameters and rerun
current models = ('All' | 'trans_cats" | 'crop_nocrop' | 'crop_nocrop_medcrop' | 'crop_nocrop_medcrop_tree' | 'veg' | 'cropType' or 'single_X' (where X is any unique string in the USE_NAME column) for binary classification of X vs all else) 

In [ ]:
rf0 = rf_model(classification_params['pixdf'],
         classification_params['model_dir'],
         classification_params['lc_mod'],
         classification_params['impmeth'],
         classification_params['ranhold'],
         classification_params['model_name'],
         lut)

## view confusion matrices
Note parameters: (pred_col, obs_col, lut, lc_mod_map, lc_mod_acc, print_cm=False, out_dir=None, model_name=None)
To print cm to csv file, change print_cm to True and provide an out_dir and model_Name

In [ ]:
#cm_cropNoCrop = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'crop_nocrop',False,None,None)
#cm_cropType = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'cropType',False,None,None)
#cm_veg = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'veg',False,None,None)
#cm_all = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'All',False,None,None)
cm_single = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],classification_params['lc_mod'],False,None,None)

#print(cm_cropNoCrop)
#print(cm_cropType)
#print(cm_veg)
#print(cm_all)
print(cm_single)

## view variable importance
this can be computed via Impurity or Permutation method (see sklearn docs)  by setting impmeth in rf_model
The full list is stored in the model directory for further manipulation 

In [ ]:
var_imp_path = os.path.join(classification_params['model_dir'],'VarImportance_{}.csv'.format(classification_params['model_name']))
var_imp = pd.read_csv(var_imp_path, names=['var','imp'], header=None)
## view 10 most important variables:
var_imp.sort_values('imp', ascending=False).head(10)

## To save an html copy of this notebook with all outputs:
(these two cells should be last in notebook)

In [ ]:
%%script echo skipping
### comment out above line and run this cell to print output as html

out_name = str(basic_config['country']+'6b_RandomFoest_ModelComparisons'+'_model'+str(classification_params['model_name'])+'_'+'Tests1')
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --ExecutePreprocessor.store_widget_state=True --output=$out_name 6b_RandomFoest_ModelComparisons.ipynb